In [1]:
import requests 
from bs4 import BeautifulSoup
import pandas as pd
import re
import urllib.parse
from tqdm.notebook import tqdm

In [3]:
def get_all_links(online_table_link):
    if online_table_link is None:
        return None
    page = requests.get(online_table_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    all_res = soup.find_all('a', {'href': re.compile(r'SEG[0-9]{3}.HTM')})
    return list(map(lambda x: urllib.parse.urljoin(online_table_link, x.get('href')), all_res))

In [41]:
def parse_officials(online_table_link, data, info):
    if online_table_link is None:
        return None
    page = requests.get(online_table_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    try:
        link = soup.find('a', {'href': re.compile(r'SEG[0-9]{3}OF.HTM')}).get('href')
    except:
        return
    if link is None:
        return None
    link = urllib.parse.urljoin(
        online_table_link, 
        link
    )
    page = requests.get(link)
    if page.status_code != 200:
        return None
    soup = BeautifulSoup(page.text, 'html.parser')
    lines = soup.find_all('tr')
    for line in lines:
        cells = line.find_all('td')
        if len(cells) > 1:
            if cells[0].text.strip() != "":
                data['function'].append(cells[0].text.strip())
                data['name'].append(cells[1].text.strip())
                for key in info:
                    data[key].append(info[key])

In [4]:
def parse_result_line(line, data):
    cells = line.find_all('td')
    if len(cells) < 10:
        return False
    rank = cells[0].text
    full_name = cells[1].a.text
    club = cells[1].text[len(full_name):]
    full_name = full_name.replace("ё", "е").replace("Ё", "Е")
    club = club.replace("ё", "е").replace("Ё", "Е")
    full_name = [x.strip() for x in full_name.split()]
    if len(full_name) == 2:
        first_name = full_name[0]
        middle_name = ""
        last_name = full_name[1]
    elif len(full_name) == 3:
        first_name = full_name[0]
        middle_name = full_name[1]
        last_name = full_name[2]
    else:
        return False
    tss = float(cells[2].text)
    tes = float(cells[3].text)
    pcs = float(cells[5].text)
    data['rank'].append(rank)
    data['firstname'].append(first_name)
    data['middlename'].append(middle_name)
    data['lastname'].append(last_name)
    data['club'].append(club)
    data['tss'].append(tss)
    data['tes'].append(tes)
    data['pcs'].append(pcs)
    return True

In [5]:
def get_category_and_segment(page):
    soup = BeautifulSoup(page.text, 'html.parser')
    if len(soup.find_all('h2')) >= 1:
        return  [x.strip() for x in soup.find_all('h2')[1].text.split(' - ')[:2]]

In [6]:
def parse_results(entries_link, data, info):
    if entries_link is None:
        return None
    page = requests.get(entries_link)
    if page.status_code != 200:
        return None
    category, segment = get_category_and_segment(page)
    soup = BeautifulSoup(page.text, 'html.parser')
    lines = soup.find_all('tr')
    for line in lines:
        if parse_result_line(line, data):
            data['category'].append(category)
            data['segment'].append(segment)
            for key in info:
                data[key].append(info[key])

In [38]:
def parse_event(event_link, data, judges):
    soup = BeautifulSoup(requests.get(event_link).text, 'html.parser')
    title = soup.find("h1", "entry-title").text
    date = soup.find("div", "competition-date").p.text.split('-')[0].strip()
    place = soup.find("div", "competition-place").p.span.text.strip()
    online = soup.find("div", "competition-file")
    if online is not None:
        online_link = online.a.get('href')
        online_link = online_link if online_link[-1] == '/' else online_link + '/'
        print(online_link)
        segments = get_all_links(online_link)
        parse_officials(online_link, judges, {'date': date, 'place': place, 'online': online_link})
        for segment in segments:
            parse_results(segment, data, {'date': date, 'place': place, 'online': online_link})

In [13]:
def parse_season(season_link, data, judges):
    page = requests.get(season_link)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    competitions = soup.find_all('tr')
    for i in tqdm(range(len(competitions))): 
        title = competitions[i].find_all('td')[1]
        if title.a is not None:
            online_link = None
            proto_link = None
            event_link = title.a.get('href')
            parse_event(event_link, data, judges)

In [42]:
for season in range(2020, 2021):
    data = {
        'date': [],
        'place': [], 
        'online':[],
        'category': [],
        'segment': [],
        'rank': [],
        'firstname': [],
        'middlename': [],
        'lastname': [],
        'club': [],
        'tss': [],
        'tes': [],
        'pcs': []
    }
    judges = {
        'date': [],
        'place': [], 
        'online':[],
        'function': [],
        'name': [],
    }
    print(f"season - {str(season)[-2:]}-{str(season+1)[-2:]}")
    parse_season(f"http://ffkkmo.ru/calendar/?season={season}", data, judges)
    df = pd.DataFrame.from_dict(data)
    df.to_csv(f"ffkkmo_{str(season)[-2:]}{str(season+1)[-2:]}_dump.csv")
    df = pd.DataFrame.from_dict(judges)
    df.to_csv(f"ffkkmo_judges_{str(season)[-2:]}{str(season+1)[-2:]}_dump.csv")

season - 20-21


  0%|          | 0/52 [00:00<?, ?it/s]

http://ffkkmo.ru/events/2021/lytkarino0927/
http://ffkkmo.ru/wp-content/uploads/2020/10/3-4-октября.pdf/
http://ffkkmo.ru/events/2021/krasnogorsk1718/
http://ffkkmo.ru/events/2021/noginsk2425/
http://ffkkmo.ru/events/2021/lytkarino1031/
http://ffkkmo.ru/events/2021/chemp0608/
http://ffkkmo.ru/events/2021/chempionat0608/
http://ffkkmo.ru/events/2021/dolgoprudniy1114/
http://ffkkmo.ru/events/2021/lobnya1611/
http://ffkkmo.ru/events/2021/lytkarino1128/
http://ffkkmo.ru/events/2021/vidnoe0112/
http://ffkkmo.ru/events/2021/spec2020/
http://ffkkmo.ru/events/2021/krasnogorsk1213/
http://ffkkmo.ru/events/2021/lytkarino1215/
http://ffkkmo.ru/events/2021/pervenstvo1719/
http://ffkkmo.ru/events/2021/sposad1226/
http://ffkkmo.ru/events/2021/lytkarino2912/
http://ffkkmo.ru/events/2021/noginsk0112/
http://ffkkmo.ru/events/2021/lytkarino0116/
http://ffkkmo.ru/events/2021/noginsk0202/
http://ffkkmo.ru/events/2021/lobnya0209/
http://ffkkmo.ru/events/2021/cfo2021/
http://ffkkmo.ru/events/2021/krasnogors